<a href="https://colab.research.google.com/github/DakshMakhija/ecommerce-cod-cleansing-analysis/blob/main/Reconzo_Internship_Assignment_%E2%80%93_Data_Cleaning_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reconzo Internship Assignment – Data Cleaning (Python)

In [29]:
!pip install xlsxwriter
import pandas as pd


In [30]:
COD_courier = pd.read_excel('/content/COD courier.xlsx', sheet_name = None)
orders = pd.read_excel('/content/orders.xlsx')
orders.head()

,Unnamed: 0,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,...,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,0,673814,NaN,pending,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,673815,NaN,pending,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,673816,NaN,paid,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,673817,NaN,pending,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,673818,NaN,voided,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cleaning orders.**xlsx**

In [31]:
# Removed empty rows
orders = orders.dropna(how='all')

# Strip whitespace from column names
orders.columns = orders.columns.str.strip()

# Check for duplicates
orders = orders.drop_duplicates()

orders['Paid at'] = pd.to_datetime(orders['Paid at'], errors='coerce')

# Removed rows where order is not valid (if any column like 'Name' or 'Financial Status' is missing)
orders = orders[orders['Name'].notna() & orders['Financial Status'].notna()]


Cleaning Each Courier Sheet

In [32]:
# dictionary to store cleaned courier sheets
cleaned_couriers = {}

for courier, df in COD_courier.items():
    print(f"\n📋 Cleaning {courier}...")

    # Dropped fully empty rows
    df = df.dropna(how='all')

    # Strip column names
    df.columns = df.columns.str.strip()

    # Drop duplicate entries
    df = df.drop_duplicates()

    for col in df.columns:
        if 'date' in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce')

    cleaned_couriers[courier] = df

    print(f"{courier} cleaned. Rows: {df.shape[0]}")



📋 Cleaning Shiprocket...
Shiprocket cleaned. Rows: 176

📋 Cleaning Ecom Xpress...
Ecom Xpress cleaned. Rows: 803

📋 Cleaning Nimbus...
Nimbus cleaned. Rows: 1205

📋 Cleaning BlueDart...
BlueDart cleaned. Rows: 12852


Saved the clean data back to orders.xlsx

In [33]:

for col in orders.select_dtypes(include=['datetimetz']).columns:
    orders[col] = orders[col].dt.tz_localize(None)

# Saved cleaned orders data
orders.to_excel('/content/orders.xlsx', index=False)


Saved clean Data back To COD courier.xlsx

In [34]:
with pd.ExcelWriter('/content/COD courier.xlsx', engine='xlsxwriter') as writer:
    for name, df in cleaned_couriers.items():
        # Copied the dataframe to avoid altering the original
        df_clean = df.copy()

        # Removed timezone info from datetime columns if present
        for col in df_clean.select_dtypes(include=['datetimetz']).columns:
            df_clean[col] = df_clean[col].dt.tz_localize(None)

        safe_name = name.strip().replace('/', '_')[:31]
       # Saved to Excel
        df_clean.to_excel(writer, sheet_name=safe_name, index=False)